<a href="https://colab.research.google.com/github/alexanderBennington/BETO_cardiologia/blob/main/cardiologia_hugg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate pandas transformers[onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install transformers[onnx]

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, pipeline, DefaultDataCollator, BertForQuestionAnswering
import torch
from evaluate import load

In [ ]:
# Cargar el archivo CSV y verificar si la ruta es correcta
try:
    datos = pd.read_csv('./cardiologia.csv', encoding="utf-8")  # Carga el archivo en un DataFrame
    print(datos.head())  # Muestra las primeras filas para verificar el contenido
except FileNotFoundError:
    print("El archivo no se encontró. Verifica la ruta.")  # Muestra un mensaje de error si el archivo no se encuentra

# Crear el dataset desde el DataFrame de pandas
dataset = Dataset.from_pandas(datos)
print(dataset)

   id_column                                            context  \
0          1  El corazón está en el centro del pecho, cerca ...   
1          2  El corazón es un órgano que tiene el tamaño ap...   
2          3  El corazón es un órgano que tiene el tamaño ap...   
3          4  Las dos cámaras superiores del corazón se llam...   
4          5  Una pared interna de tejido divide los lados d...   

                                            question  \
0                            ¿Dónde esta el corazón?   
1                                ¿Qué es el corazón?   
2                   ¿Cuál es la función del corazón?   
3  ¿Cómo se llaman las camaras superiores del cor...   
4                                ¿Qué es el tabique?   

                                              answer  start_positions  \
0      en el centro del pecho, cerca de los pulmones               16   
1  es un órgano que tiene el tamaño aproximado de...               11   
2                   bombea sangre por tod

In [ ]:
# Validar datos y limpiar valores nulos
datos = datos.dropna(subset=['context', 'question', 'answer', 'start_positions'])
dataset = Dataset.from_pandas(datos)
print(dataset)

Dataset({
    features: ['id_column', 'context', 'question', 'answer', 'start_positions', 'end_positions'],
    num_rows: 225
})


In [ ]:
# Preparar los datos en el formato adecuado para el modelo de pregunta-respuesta
def preparar_datos(datos):
    return {
        'context': datos['context'],                    # Extrae el contexto
        'question': datos['question'],                  # Extrae la pregunta
        'answers': {                                    # Crea un diccionario para la respuesta
            'text': [datos['answer']],                  # Incluye la respuesta como lista
            'answer_start': [datos['start_positions']]  # Posición de inicio de la respuesta en el contexto
        }
    }

# Aplica la función a cada entrada del dataset
dataset = dataset.map(preparar_datos)
print(dataset)

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Dataset({
    features: ['id_column', 'context', 'question', 'answer', 'start_positions', 'end_positions', 'answers'],
    num_rows: 225
})


In [ ]:
print(dataset[0])

{'id_column': 1, 'context': 'El corazón está en el centro del pecho, cerca de los pulmones', 'question': '¿Dónde esta el corazón?', 'answer': 'en el centro del pecho, cerca de los pulmones', 'start_positions': 16, 'end_positions': 61, 'answers': {'answer_start': [16], 'text': ['en el centro del pecho, cerca de los pulmones']}}


In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
tokenizador = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
modelo = AutoModelForQuestionAnswering.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def funcion_preproceso(datos):
    questions = [q.strip() for q in datos["question"]]
    inputs = tokenizador(
        questions,
        datos["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = datos["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i) # identifica si un token pertenece a la pregunta, al contexto o al padding.

        # Encuentra las posiciones de inicio y fin. Identifica los índices donde empieza (context_start) y termina (context_end) el contexto en la secuencia tokenizada.
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Si la respuesta no está completamente dentro del contexto, marcar como (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # De lo contrario, son las posiciones de token inicial y final.
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1) #Avanza hasta encontrar el token cuyo inicio coincide con el inicio de la respuesta.

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1) #Retrocede hasta encontrar el token cuyo final coincide con el final de la respuesta.

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
dataset_tokenizado = dataset.map(funcion_preproceso, batched=True)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [ ]:
print(dataset_tokenizado)

DatasetDict({
    train: Dataset({
        features: ['id_column', 'context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 180
    })
    test: Dataset({
        features: ['id_column', 'context', 'question', 'answer', 'start_positions', 'end_positions', 'answers', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 45
    })
})


In [ ]:
#print(dataset_tokenizado[0])
# Obtiene los IDs de tokens de un ejemplo y conviértelos a palabras legibles
ejemplo_tokens = dataset_tokenizado["train"][0]['input_ids']
print(tokenizador.convert_ids_to_tokens(ejemplo_tokens),'\n')
print(dataset_tokenizado["train"][4]['input_ids'],'\n')#Son IDs que corresponden a cada token, son los valores numéricos que el modelo utiliza internamente en lugar de las palabras.
print(dataset_tokenizado["train"][4]['attention_mask'],'\n')#Tiene valores 1 para los tokens relevantes (los de la pregunta y contexto) y 0 para el padding
print(dataset_tokenizado["train"][4]['start_positions'])

['[CLS]', '¿', 'Qué', 'posibili', '##tan', 'todas', 'las', 'partes', 'del', 'sistema', 'respira', '##torio', '?', '[SEP]', 'El', 'sistema', 'respira', '##torio', 'también', 'incluye', 'la', 'trá', '##que', '##a', ',', 'los', 'músculos', 'de', 'la', 'pared', 'tor', '##ác', '##ica', 'y', 'el', 'dia', '##fra', '##g', '##ma', ',', 'los', 'vasos', 'sanguíneo', '##s', 'y', 'otros', 'tejidos', '.', 'Todas', 'esas', 'partes', 'posibili', '##tan', 'la', 'respiración', 'y', 'el', 'intercambio', 'de', 'gases', '.', 'El', 'cerebro', 'controla', 'la', 'frecuencia', 'respira', '##toria', '(', 'que', 'tan', 'rápido', 'o', 'lento', 'respira', '##mos', ')', 'al', 'identificar', 'la', 'necesidad', 'de', 'oxígeno', 'del', 'cuerpo', 'y', 'también', 'la', 'necesidad', 'de', 'eliminar', 'dióxido', 'de', 'carbono', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[

In [ ]:
print(tokenizador.convert_ids_to_tokens(4),'\n')

[CLS] 



In [ ]:
data_collator = DefaultDataCollator() #prepara lotes (batches) de datos que puedan ser procesados eficientemente en paralelo durante el entrenamiento o la evaluación

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Función para calcular "Exact Match" (EM)
def exact_match(prediction, reference):
    return int(prediction.strip() == reference.strip())

# Función para calcular "F1 Score"
def f1_score_metric(prediction, reference):
    pred_tokens = prediction.split()
    ref_tokens = reference.split()
    common = set(pred_tokens) & set(ref_tokens)
    if len(common) == 0:
        return 0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1
def compute_metrics(p):
    predictions = p.predictions
    references = p.label_ids

    # Decodificar las respuestas predichas y las reales
    pred_texts = []
    for batch in predictions:
        pred_token_ids = [np.argmax(pred) for pred in batch]
        pred_texts.extend([tokenizer.decode(pred_id, skip_special_tokens=True) for pred_id in pred_token_ids])

    ref_texts = [tokenizer.decode(ref, skip_special_tokens=True) for ref in references]

    exact_match_scores = [exact_match(pred, ref) for pred, ref in zip(pred_texts, ref_texts)]
    f1_scores = [f1_score_metric(pred, ref) for pred, ref in zip(pred_texts, ref_texts)]

    # Promediar los puntajes
    exact_match_avg = sum(exact_match_scores) / len(exact_match_scores)
    f1_avg = sum(f1_scores) / len(f1_scores)

    return {
        "exact_match": exact_match_avg,
        "f1": f1_avg,
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",                           #Especifica el directorio donde se guardarán los resultados del entrenamiento
    eval_strategy="epoch",                            #Indica la estrategia de evaluación "epoch" significa que el modelo se evaluará automáticamente al final de cada época.
    learning_rate=2e-5,                               #Especifica la tasa de aprendizaje, que es un parámetro crítico para el ajuste de pesos en cada paso de entrenamiento.
    per_device_train_batch_size=8,                    #Define el tamaño del lote (batch) de datos para entrenamiento y evaluación, respectivamente, en cada GPU o CPU disponible.
    per_device_eval_batch_size=8,                     #Un tamaño de lote pequeño permite reducir el uso de memoria, aunque puede afectar la estabilidad de los gradientes.
    num_train_epochs=3,                               #Una época completa significa que el modelo verá todo el conjunto de entrenamiento una vez.
    weight_decay=0.01,                                #El "decaimiento de los pesos" es una forma de regularización para prevenir el sobreajuste.
    report_to="none",                                 #Controla dónde se reportan las métricas de entrenamiento.
    logging_dir='./logs',                             #Define el directorio para almacenar los registros del entrenamiento, como la pérdida (loss) y otras métricas.
    logging_steps=50,                                 #Cada 50 pasos de entrenamiento, el Trainer registrará las métricas de entrenamiento en los archivos de registro
)

trainer = Trainer(
    model=modelo,
    args=training_args,
    train_dataset=dataset_tokenizado["train"],
    eval_dataset=dataset_tokenizado["test"],
    tokenizer=tokenizador,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Exact Match,F1
1,No log,2.558859,0.000000,0.021739
2,No log,2.328861,0.000000,0.021739
3,0.693000,2.463017,0.000000,0.021739


TrainOutput(global_step=69, training_loss=0.6784583520198214, metrics={'train_runtime': 46.3133, 'train_samples_per_second': 11.66, 'train_steps_per_second': 1.49, 'total_flos': 105825186478080.0, 'train_loss': 0.6784583520198214, 'epoch': 3.0})

In [ ]:
# Guardar el modelo y tokenizer
modelo.save_pretrained('./modelo-finetuned-cardiologia2')
tokenizador.save_pretrained('./modelo-finetuned-cardiologia2')

('./modelo-finetuned-cardiologia2/tokenizer_config.json',
 './modelo-finetuned-cardiologia2/special_tokens_map.json',
 './modelo-finetuned-cardiologia2/vocab.txt',
 './modelo-finetuned-cardiologia2/added_tokens.json',
 './modelo-finetuned-cardiologia2/tokenizer.json')

In [ ]:
# Cargar el contenido de un archivo .txt como contexto
try:
    with open("contexto.txt", "r", encoding="utf-8") as file:
        context = file.read()
except UnicodeDecodeError:
    # Si falla utf-8, intenta con 'latin-1'
    with open("contexto.txt", "r", encoding="latin-1") as file:
        context = file.read()

In [ ]:
context="""El corazón es un órgano que tiene el tamaño aproximado de un puño y que bombea sangre por todo el cuerpo. El colesterol es una sustancia cerosa y gorosa que se encuentra en todas las células del cuerpo.
El colesterol alto en sangre aumenta el riesgo de desarrollar cardiopatía coronaria. La arteria es Un vaso sanguíneo que transporta sangre desde el corazón hasta los órganos. La mayoría de las arterias transportan sangre oxigenada.
Las arterias pulmonares llevan la sangre desoxigenada del corazón a los pulmones. La placa es una sustancia compuesta de grasa, colesterol, calcio y otras sustancias que se acumula y endurece en las arterias, lo que reduce el flujo sanguíneo.
Un coágulo de sangre es una aglomeración similar a un gel que se forma cuando la sangre cambia de un estado líquido a un estado sólido. Los coágulos de sangre son necesarios para ayudar a detener el sangrado después de una lesión.
Un coágulo de sangre anormal que se forma dentro de una arteria o vena se llama trombo. Una parte del coágulo de sangre, llamada émbolo, puede desprenderse y moverse a través del torrente sanguíneo.
El émbolo puede bloquear el flujo sanguíneo en otra parte del cuerpo, como el cerebro o los pulmones"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")

In [ ]:
question = input("Ingrese su pregunta: ")
inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():#Desactiva el cálculo de gradientes (que no se necesita para la inferencia), lo cual reduce el uso de memoria y acelera el proceso.
    outputs = model(**inputs)#Ejecuta el modelo para obtener las predicciones de inicio y fin de la respuesta dentro del contexto. outputs contiene start_logits y end_logits, que son vectores que indican la probabilidad de que cada token sea el inicio o el fin de la respuesta.
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]#Extrae los tokens correspondientes a la respuesta, desde answer_start_index hasta answer_end_index, usando input_ids. input_ids contiene los tokens de la pregunta y el contexto en la entrada.
tokenizer.decode(predict_answer_tokens)

Ingrese su pregunta: que es trombo?


'Un coágulo de sangre anormal que se forma dentro de una arteria o vena'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Cambia la ruta a donde deseas guardar el modelo en tu Google Drive
drive_path = '/content/drive/My Drive/modelo-finetuned-cardiologia0_678'

# Define la ruta local del modelo
local_model_path = './modelo-finetuned-cardiologia2'

# Copia el modelo y el tokenizador a Google Drive
# Agrega el argumento dirs_exist_ok=True para permitir la copia si el directorio existe
shutil.copytree(local_model_path, drive_path, dirs_exist_ok=True)

'/content/drive/My Drive/modelo-finetuned-cardiologia0_678'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")

In [ ]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
# Cargar el contenido de un archivo .txt como contexto
try:
    with open("contexto.txt", "r", encoding="utf-8") as file:
        context = file.read()
except UnicodeDecodeError:
    # Si falla utf-8, intenta con 'latin-1'
    with open("contexto.txt", "r", encoding="latin-1") as file:
        context = file.read()

In [ ]:
def respuesta(question):
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens)

In [ ]:
question = input("Ingrese su pregunta: ")
print(respuesta(question))

Ingrese su pregunta: que es el corazon?
es un órgano que tiene el tamaño aproximado de un puño y que bombea sangre por todo el cuerpo


In [ ]:
model_name = "modelo-finetuned-cardiologia0_678"
model_path = "./" + model_name + ".onnx"
model = BertForQuestionAnswering.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
inputs = {
        #Una lista de IDs de tokens (aquí, valores aleatorios) con tamaño [1, 32], lo que simula una secuencia tokenizada con 32 tokens.
        'input_ids':   torch.randint(32, [1, 32], dtype=torch.long),
        #Una máscara de atención de unos con tamaño [1, 32], que indica que todos los tokens deben ser considerados.
        'attention_mask': torch.ones([1, 32], dtype=torch.long),
        #Una lista de unos de tamaño [1, 32] que designa el tipo de token (por ejemplo, si pertenece a la pregunta o al contexto en modelos de QA)
        'token_type_ids':  torch.ones([1, 32], dtype=torch.long)
    }

symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
torch.onnx.export(model,
# model being run
                  (inputs['input_ids'],
                   inputs['attention_mask'],
                   inputs['token_type_ids']),                    # Entradas del modelo
                  model_path,                                    # Ruta de guardado
                  opset_version=19,                              # Versión de ONNX
                  do_constant_folding=True,                      # Optimización de constantes
                  input_names=['input_ids',
                               'input_mask',
                               'segment_ids'],                   # Nombres de las entrada
                  output_names=['start_logits', "end_logits"],   # Nombres de las salidas
                  dynamic_axes={'input_ids': symbolic_names,
                                'input_mask' : symbolic_names,
                                'segment_ids' : symbolic_names,
                                'start_logits' : symbolic_names,
                                'end_logits': symbolic_names})


In [4]:
import onnxruntime as ort
import numpy as np
from transformers import AutoTokenizer

# Cargar el modelo ONNX y el tokenizador
model_path = "./modelo-finetuned-cardiologia0_678.onnx"
tokenizer = AutoTokenizer.from_pretrained("/content/drive/My Drive/modelo-finetuned-cardiologia0_678")

# Crear una sesión de ONNX Runtime, InferenceSession carga el modelo y prepara los recursos necesarios para inferencias rápidas y eficientes.
ort_session = ort.InferenceSession(model_path)

# Definir la función de inferencia con ONNX
def respuesta(question, context):
    # Tokenizar la entrada
    inputs = tokenizer(question, context, return_tensors="np", padding="max_length", max_length=384, truncation=True)

    # Extraer los datos de entrada para el modelo ONNX
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    # Realizar la inferencia con ONNX Runtime
    #ort_session.run() toma como argumentos las salidas deseadas (start_logits y end_logits) y las entradas necesarias (input_ids, input_mask, segment_ids).
    #start_logits y end_logits representan las probabilidades de cada posición para ser el inicio y el final de la respuesta, respectivamente.
    outputs = ort_session.run(
        ["start_logits", "end_logits"],
        {
            "input_ids": input_ids,
            "input_mask": attention_mask,
            "segment_ids": token_type_ids,
        }
    )

    # Obtener las posiciones de inicio y fin de la respuesta
    #Usa np.argmax para encontrar las posiciones de los índices con mayor probabilidad en start_logits y end_logits, indicando el inicio y fin de la respuesta.
    start_logits = outputs[0]
    end_logits = outputs[1]
    answer_start_index = np.argmax(start_logits)
    answer_end_index = np.argmax(end_logits)

    # Decodificar los tokens de la respuesta
    #predict_answer_tokens selecciona los tokens de input_ids entre answer_start_index y answer_end_index + 1.
    #tokenizer.decode() convierte los tokens a texto natural, excluyendo tokens especiales con skip_special_tokens=True.
    predict_answer_tokens = input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# Ejemplo de uso
question = "que transportan las arterias??"
context = """El corazón es un órgano que tiene el tamaño aproximado de un puño y que bombea sangre por todo el cuerpo. El colesterol es una sustancia cerosa y gorosa que se encuentra en todas las células del cuerpo.
El colesterol alto en sangre aumenta el riesgo de desarrollar cardiopatía coronaria. La arteria es un vaso sanguíneo que transporta sangre desde el corazón hasta los órganos. La mayoría de las arterias transportan sangre oxigenada.
Las arterias pulmonares llevan la sangre desoxigenada del corazón a los pulmones. La placa es una sustancia compuesta de grasa, colesterol, calcio y otras sustancias que se acumula y endurece en las arterias, lo que reduce el flujo sanguíneo.
Un coágulo de sangre es una aglomeración similar a un gel que se forma cuando la sangre cambia de un estado líquido a un estado sólido. Los coágulos de sangre son necesarios para ayudar a detener el sangrado después de una lesión.
Un coágulo de sangre anormal que se forma dentro de una arteria o vena se llama trombo. Una parte del coágulo de sangre, llamada émbolo, puede desprenderse y moverse a través del torrente sanguíneo.
El émbolo puede bloquear el flujo sanguíneo en otra parte del cuerpo, como el cerebro o los pulmones"""

# Imprimir la respuesta
print("Pregunta:", question)
print("Respuesta:", respuesta(question, context))


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from ./modelo-finetuned-cardiologia0_678.onnx failed:Load model ./modelo-finetuned-cardiologia0_678.onnx failed. File doesn't exist